## Библиотеки

In [ ]:
import os

import pandas as pd
import warnings
import random
from datetime import datetime


from lxml import html
import requests

import aiohttp
import asyncio
from aiohttp import ClientTimeout
from aiohttp_socks import ProxyConnector
from aiohttp import ClientSession
from stem import Signal
from stem.control import Controller

from tqdm.asyncio import tqdm
from charset_normalizer import from_bytes

from datetime import datetime as Ddtime
from datetime import timedelta

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore")

## Функции

In [ ]:
REQUEST_AMOUNT = 10 # больше 10 - плохая идея

semaphore = asyncio.Semaphore(REQUEST_AMOUNT)

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 Edg/93.0.961.52",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
]

LANGUAGES = [
    "en-US,en;q=0.9",
    "en-GB,en;q=0.9",
    "fr-FR,fr;q=0.9",
    "de-DE,de;q=0.9",
    "es-ES,es;q=0.9",
    "ru-RU,ru;q=0.9",
    "zh-CN,zh;q=0.9",
    "ja-JP,ja;q=0.9",
    "it-IT,it;q=0.9",
    "pt-BR,pt;q=0.9",
]

TOR_PROXY = "socks5://127.0.0.1:9150"  # Порт Tor Browser
CONTROL_PORT = 9051  # Порт ControlPort
CONTROL_PASSWORD = "ENTER_YOUR_OWN_PASSWORD"

ip_change_lock = asyncio.Lock()

def isNumber(s):
    negative = False
    if(s[0] =='-'):
        negative = True
         
    if negative == True:
        s = s[1:]
    try:
        n = int(s)
        return True
    except ValueError:
        return False

### Функция для обновления IP внутри Тора (используется для
### преодоления 429 ответа от сайтов и прочих неприятностей)
def renew_tor_ip():
    with Controller.from_port(port=CONTROL_PORT) as controller:
        controller.authenticate(password=CONTROL_PASSWORD)
        controller.signal(Signal.NEWNYM)

### Основная функция для асинхронного скачивания HTML-кода страницы
### запрашиваемого ресурса с внедренным обработчиком ошибок
async def fetch(session, home_url, page_num, max_retries=10):
    retries = 0
    while retries < max_retries:
        if home_url.find("?") != -1:
            st = home_url[:home_url.find("?")]
            url = st + "/"*int(st[-1] != "/") + f'page{page_num}/' + home_url[home_url.find("?"):]
        else:
            url = home_url + "/"*int(home_url[-1] != "/") + f'page{page_num}/'
        try:
            async with semaphore:
                await asyncio.sleep(1)
                headers = {
                    "User-Agent": random.choice(USER_AGENTS),
                    "Accept-Language": random.choice(LANGUAGES),
                    "Referer": "https://example.com",
                }
                async with session.get(url, headers=headers) as response:
                    if response.status == 200:
                        content = await response.content.read()
                        en = from_bytes(content)
                        encoding = en.best().encoding
                        return content.decode(encoding)
                    elif response.status == 429:  # Too Many Requests
                        retry_after = response.headers.get("Retry-After")
                        if retry_after:
                            try:
                                wait_time = int(retry_after)
                            except ValueError:
                                retry_time = datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
                                wait_time = (retry_time - datetime.now()).total_seconds()
                            await asyncio.sleep(wait_time)
                        else:
                            async with ip_change_lock:
                                renew_tor_ip()
                                await asyncio.sleep(15)
                    else:
                        async with ip_change_lock:
                            print(f"Server returned {response.status} for {url}. Changing IP...")
                            renew_tor_ip()
                            await asyncio.sleep(15)
                    retries += 1
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            print(f"Error fetching {url}: {e}")
            retries += 1
            if retries < max_retries:
                async with ip_change_lock:
                    print("Changing IP due to error...")
                    renew_tor_ip()
                    await asyncio.sleep(15)
    return None

### Функция-обработчик открытой страницы: возвращает
### pd.DataFrame со всеми размещенными на ней объявлениями
async def process_page(home_url, page_num, session):
    content = await fetch(session, home_url, page_num)
    if type(content) != type(None):
        tree = html.fromstring(content)

        divs = tree.xpath('//div[@data-ftid="bulls-list_bull"]')

        urls = []
        car_labels = []
        car_year = []

        df = pd.DataFrame({"Название машины": car_labels, "Год": car_year, "Ссылка": urls})

        if len(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
            return df

        h2 = tree.xpath('//div[@data-ftid="bull_date"]')
        RUSS = {"дек": "12",
                "янв": "01",
                "фев": "02",
                "мар": "03",
                "апр": "04",
                "июн": "06",
                "июл": "07",
                "авг": "08",
                "сен": "09",
                "окт": "10",
                "ноя": "11",
                }

        for i, div in enumerate(divs):
            links = div.xpath('.//a')
            h3 = div.xpath('.//h3')
            
            car_labels.append(h3[0].text_content().split(", ")[0].strip())
            car_year.append(h3[0].text_content().split(", ")[1].strip())
            
            for link in links[:1]:
                href = link.get('href')
                urls.append(href)


        date = [h2[i].text_content().split() for i in range(len(h2))]
        for i in range(len(date)):
            if len(date[i]) == 2 or not(isNumber(date[i][2])):
                date[i] = date[i] + ["2024"]
            month = date[i][1]
            if len(month) == 3:
                date[i] = date[i][0] + ".05." + date[i][2]
            else:
                m = month[:3]
                if m == "час":
                    date[i] = Ddtime.now() - timedelta(hours = int(date[i][0]))
                else:
                    date[i] = date[i][0] + f".{RUSS[m]}." + date[i][2]
        date = pd.to_datetime(pd.Series(date), format = "%d.%m.%Y")

        df = pd.DataFrame({"Название машины": car_labels, "Год": car_year, "Ссылка": urls, "Дата размещения объявления": date})
        df["Дата размещения объявления"] = df["Дата размещения объявления"].dt.round("1d")

        return df
    return None

### Базовый обработчик для асинхронного запроса
async def worker(url_queue, session, results, pbar):
    while True:
        mas = await url_queue.get()
        if mas is None:
            break
        else:
            home_url, page_num = mas
        try:
            data = await process_page(home_url, page_num, session)
            if type(data) != type(None):
                results.append(data)
        except Exception as e:
            print(f"Error processing URL {home_url, page_num}: {e}")
        finally:
            url_queue.task_done()
            pbar.update(1)

### Функция для вызова
async def main(urls, text):
    url_queue = asyncio.Queue()
    results = []

    for url in urls:
        for page_num in range(1, 20 + 1):
            url_queue.put_nowait((url, page_num))

    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        with tqdm(total=len(urls) * 20, desc=f"Обработка файла {text}") as pbar:
            workers = [asyncio.create_task(worker(url_queue, session, results, pbar)) for _ in range(REQUEST_AMOUNT)]
            
            await url_queue.join()

            for _ in range(REQUEST_AMOUNT):
                await url_queue.put(None)

            await asyncio.gather(*workers)

    return results

## Код

In [ ]:
files = []
for filename in os.listdir(r"./cars/href/"):
    if filename.endswith("car.xlsx"):
        files.append(filename)
len(files)
files = sorted(files, key= lambda x: int(x[:x.find("car")]))
starting = 0

In [ ]:
for j in range(starting, len(files)):
    starting = j
    car_name = files[j]
    valid_urls = list(pd.read_excel(f"./cars/href/{car_name}")["Ссылка"])
    car_df = pd.DataFrame({"Название машины": [], "Год": [], "Ссылка": [], "Дата размещения объявления": []})
    print(f"Количество валидных категорий для сбора данных: {len(valid_urls)}")
    
    res = await main(valid_urls, car_name)

    for dff in res:
        car_df = pd.concat([car_df, dff], axis = 0)

    car_df.reset_index(inplace = True, drop = True)
    car_df.drop_duplicates(inplace= True)
    print(f"Количество собранных объявлений: {len(car_df)}")
    name = car_name[:car_name.find("car")] + "first"
    car_df.to_excel(f"./cars/finished/{name}.xlsx", index = False)
    print(f"Файл сохранен как {name}")
    print(("="*80 + "\n"*1))

In [ ]:
### Пример подготовки к вторичному парсингу

ultra = pd.read_excel(f"./cars/finished/0first.xlsx")

for filename in os.listdir(r"./cars/finished/"):
    if filename.endswith("first.xlsx"):
        df = pd.read_excel(f"./cars/finished/{filename}")
        ultra = pd.concat([ultra, df], axis = 0)

ultra.drop_duplicates(inplace=True)
ultra.reset_index(inplace=True, drop=True)
print("Всего объявлений спаршено: ", len(ultra))

ultra["Дата размещения объявления"] = pd.to_datetime(ultra["Дата размещения объявления"])
new_ultra = ultra[(ultra["Дата размещения объявления"] >= Ddtime(year=2018, day=1, month=1))\
      & (ultra["Дата размещения объявления"] < Ddtime(year=2024, day=1, month=1))].reset_index(inplace=False, drop = True)
new_ultra.drop_duplicates(inplace=True)
new_ultra.reset_index(inplace=True, drop=True)
print("Всего объявлений (с учетом корректной даты): ", len(new_ultra))

i = 0
while i < len(new_ultra):
    ongoing = new_ultra.loc[i:i+999]
    ongoing.to_excel(f"./second_href/{i//1000}second.xlsx", index = False)
    i += 1000
print("Все готово к вторичному парсингу!")